System Installation

In [1]:
# 1. Setup Locale
!sudo apt update && sudo apt install locales
!sudo locale-gen en_US en_US.UTF-8
!sudo update-locale LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8

# 2. Add ROS 2 Repository
!sudo apt install software-properties-common -y
!sudo add-apt-repository universe -y
!sudo apt update && sudo apt install curl -y
!sudo curl -sSL https://raw.githubusercontent.com/ros/rosdistro/master/ros.key -o /usr/share/keyrings/ros-archive-keyring.gpg
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/ros-archive-keyring.gpg] http://packages.ros.org/ros2/ubuntu $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/ros2.list > /dev/null

# 3. Install ROS 2 Humble and Gazebo (Fortress)
!sudo apt update
!sudo apt install ros-humble-ros-base ros-humble-gazebo-ros-pkgs -y

# 4. Install Colcon (Build tool)
!sudo apt install python3-colcon-common-extensions -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,300 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,511 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports In

Creating the Robot File

In [3]:
urdf_content = """
<?xml version="1.0"?>
<robot name="ethiopia_bot">
  <link name="base_link">
    <visual><geometry><box size="0.4 0.3 0.1"/></geometry></visual>
    <collision><geometry><box size="0.4 0.3 0.1"/></geometry></collision>
    <inertial><mass value="5.0"/><inertia ixx="0.1" ixy="0" ixz="0" iyy="0.1" iyz="0" izz="0.1"/></inertial>
  </link>

  <link name="left_wheel">
    <visual><geometry><cylinder length="0.05" radius="0.1"/></geometry></visual>
  </link>
  <joint name="left_wheel_joint" type="continuous">
    <parent link="base_link"/><child link="left_wheel"/><origin xyz="-0.1 0.17 0" rpy="1.57 0 0"/><axis xyz="0 0 1"/>
  </joint>

  <link name="right_wheel">
    <visual><geometry><cylinder length="0.05" radius="0.1"/></geometry></visual>
  </link>
  <joint name="right_wheel_joint" type="continuous">
    <parent link="base_link"/><child link="right_wheel"/><origin xyz="-0.1 -0.17 0" rpy="1.57 0 0"/><axis xyz="0 0 1"/>
  </joint>

  <link name="caster_wheel">
    <visual><geometry><sphere radius="0.05"/></geometry></visual>
  </link>
  <joint name="caster_joint" type="fixed">
    <parent link="base_link"/><child link="caster_wheel"/><origin xyz="0.15 0 -0.05"/>
  </joint>

  <gazebo reference="base_link">
    <sensor name="camera" type="camera">
      <camera><horizontal_fov>1.047</horizontal_fov><image><width>640</width><height>480</height></image></camera>
      <plugin name="camera_controller" filename="libgazebo_ros_camera.so"/>
    </sensor>
  </gazebo>
</robot>
"""
with open("robot.urdf", "w") as f:
    f.write(urdf_content)
print("Robot URDF created!")

Robot URDF created!


The Search Algorithm (The Brain)

In [4]:
import heapq

# The Graph from your Figure 5 Image
ethiopia_graph = {
    'Addis Ababa': [('Adama', 99), ('Ambo', 114), ('Debre Berhan', 120)],
    'Adama': [('Addis Ababa', 99), ('Batu', 92), ('Asella', 75)],
    'Ambo': [('Addis Ababa', 114), ('Nekemte', 139), ('Wolisso', 130)],
    'Nekemte': [('Ambo', 139), ('Gimbi', 126), ('Bedele', 105)],
    'Jimma': [('Wolisso', 184), ('Bonga', 110), ('Bedele', 76)],
    'Wolisso': [('Ambo', 130), ('Addis Ababa', 135), ('Jimma', 184)],
    'Batu': [('Adama', 92), ('Shashemene', 169)],
    'Shashemene': [('Batu', 169), ('Hawassa', 25), ('Dodola', 102)],
    # ... you can keep adding pairs from your image here
}

def a_star_search(graph, start, goal):
    # This finds the path with the lowest total distance
    priority_queue = [(0, start, [])]
    visited = set()

    while priority_queue:
        (cost, current_node, path) = heapq.heappop(priority_queue)

        if current_node in visited:
            continue

        path = path + [current_node]
        if current_node == goal:
            return path, cost

        visited.add(current_node)

        for (neighbor, distance) in graph.get(current_node, []):
            if neighbor not in visited:
                heapq.heappush(priority_queue, (cost + distance, neighbor, path))
    return None, 0

# Example: Find path from Addis Ababa to Jimma
path, total_distance = a_star_search(ethiopia_graph, 'Addis Ababa', 'Jimma')
print(f"Optimal Path: {' -> '.join(path)}")
print(f"Total Distance: {total_distance} km")

Optimal Path: Addis Ababa -> Ambo -> Wolisso -> Jimma
Total Distance: 428 km


Launching Gazebo
This starts the physics simulation.

In [8]:
# Start Gazebo in headless mode (no GUI)
!source /opt/ros/humble/setup.bash && gazebo --verbose -s libgazebo_ros_factory.so robot.urdf &

Gazebo multi-robot simulator, version 11.10.2
Copyright (C) 2012 Open Source Robotics Foundation.
Released under the Apache 2 License.
http://gazebosim.org

[Msg] Waiting for master.
Gazebo multi-robot simulator, version 11.10.2
Copyright (C) 2012 Open Source Robotics Foundation.
Released under the Apache 2 License.
http://gazebosim.org

[Msg] Waiting for master.
[Msg] Connected to gazebo master @ http://127.0.0.1:11345
[Msg] Publicized address: 172.28.0.12
[Err] [RenderEngine.cc:749] Can't open display: 
[Wrn] [RenderEngine.cc:89] Unable to create X window. Rendering will be disabled
[Wrn] [RenderEngine.cc:292] Cannot initialize render engine since render path type is NONE. Ignore this warning ifrendering has been turned off on purpose.
[Msg] Loading world file [/content/robot.urdf]
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func